In [1]:
#import dependencies
import pandas as pd
import numpy as np
import requests
import json
from config import gkey

In [2]:
#dexter's code
#read in the large file
df =  pd.read_csv('./resources/data/source/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv')

#filter the years and where the origin is AUS
df = df[(df['Year'] >= 2007) & (df['Year'] <= 2017) & (df['airport_1'] == 'AUS')]

#per the dataset documentation, the column passengers holds the value for passengers per day
#the data is provided on a per quarter basis
#so the real value of the number of passengers should be the multiplied by the number of days in a quarter, or by 91 days
df['passengers'] = df['passengers'] * 91

#save this condensed dataset
df.to_csv('./resources/data/transformed/AUS.csv', index=False)

In [3]:
#dexter's code
#get the list of destination_airports to query for lat and lng
destination_airports = df['airport_2'].unique().tolist()

#get the lat and lng from the google geocode api
coords = []
for i in range(len(destination_airports)):
    coord = {}
    coord['airport_2'] = destination_airports[i]
    
    # Build the endpoint URL
    target_url = (f'https://maps.googleapis.com/maps/api/geocode/json?address={destination_airports[i]}&key={gkey}')
    
    #get the json data
    geo_data = requests.get(target_url).json()
    
    #get the lat and the lng
    coord['lat']= geo_data["results"][0]["geometry"]["location"]["lat"]
    coord['lng'] = geo_data["results"][0]["geometry"]["location"]["lng"]    
    coords.append(coord)

#save this dataset to a csv
pd.DataFrame(coords).to_csv('./resources/data/transformed/destination_airports_lat_lng.csv', index=False)

In [4]:
#Mary's code
#read in the large file
df = pd.read_csv('./resources/data/source/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv')

#filter the data, select the columns we need and save for use in the analysis notebook
df.loc[df['city1'] == 'Austin, TX'][['Year', 'quarter', 'city1', 'carrier_lg']].to_csv('./resources/data/transformed/airlinedata.csv', index=False)


In [5]:
#Alan's code

#read in the file and just pull out 2 columns
df = pd.read_csv("./resources/data/source/AUS_monthly_passengers.csv")[['Month', 'Total Passengers']]

#convert the Month to a datetime value
df['Month'] = pd.to_datetime(df['Month'])

#sort the data and save it as a new file
df.sort_values(by='Month').set_index('Month').reset_index().to_csv('./resources/data/transformed/AUS_monthly_passengers_clean.csv', index=False)

